<a href="https://colab.research.google.com/github/MariannaJan/NLTKChatbot/blob/master/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simple chatbot with nltk on the basis of [this article](https://medium.com/analytics-vidhya/building-a-simple-chatbot-in-python-using-nltk-7c8c8215ac6e)

In [1]:
#Importing the input file with the info for chatbot

#!wget https://www.dropbox.com/s/pkmhv606jjejemk/grimm.txt?dl=0
!wget https://www.dropbox.com/s/6tfdsdtuqt7qqu6/chatbot.txt?dl=0

--2019-02-02 09:52:19--  https://www.dropbox.com/s/6tfdsdtuqt7qqu6/chatbot.txt?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/6tfdsdtuqt7qqu6/chatbot.txt [following]
--2019-02-02 09:52:19--  https://www.dropbox.com/s/raw/6tfdsdtuqt7qqu6/chatbot.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd2e2a32dfcf234b650476e7a02.dl.dropboxusercontent.com/cd/0/inline/Aaixbm55BTxqykThaO6j0wEBVh8mrg74KXWZjOlpZ_hekmZD-_R-tfaRglFWhjMvXUhKvjEQChdh5Gtv-iRtw2SCyCrCjqmDBZbyuhVPjj68Vg/file# [following]
--2019-02-02 09:52:20--  https://ucd2e2a32dfcf234b650476e7a02.dl.dropboxusercontent.com/cd/0/inline/Aaixbm55BTxqykThaO6j0wEBVh8mrg74KXWZjOlpZ_hekmZD-_R-tfaRglFWhjMvXUhKvjEQChdh5Gtv-iRtw2SCyCrCjqmDBZbyuhVPjj68Vg/file
Resolving ucd2e2a

In [0]:
#Importing the natural language toolkit and libraries for text preprocessing

import nltk
import numpy as np
import random
import string # to process standard python strings

In [3]:
#Downloading the nltk tokenizer and dictionary

nltk.download('punkt') # first-time use only
nltk.download('wordnet') # first-time use only

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
#reading the input file and preprocessing it

path = "/content/chatbot.txt?dl=0"
#path = "/content/grimm.txt?dl=0"

f=open(path,'r',errors = 'ignore')
raw=f.read()
raw=raw.lower()# converts to lowercase
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

In [0]:
#Performing lemmatization - finding synoninms and words from one word family

lemmer = nltk.stem.WordNetLemmatizer()

#WordNet is a semantically-oriented dictionary of English included in NLTK.

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [0]:
#Setting up the greetings accepted by chatbot and greeting responses

GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [0]:
# Importing the libraries for normalizing the importance of word on the basis of it's frequency 
# and for finding semantic similarity of words based on word vectors

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
#Preparing the response based on the semantic similarity of the question and parts of text fro the input info text

def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [11]:
#Script for interactiong with the chatbot

flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
ROBO: hello


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


ROBO: a chatbot (also known as a smartbot, talkbot, chatterbot, bot, im bot, interactive agent, conversational interface or artificial conversational entity) is a computer program or an artificial intelligence which conducts a conversation via auditory or textual methods.
ROBO: the term "chatterbot" was originally coined by michael mauldin (creator of the first verbot, julia) in 1994 to describe these conversational programs.
ROBO: I am sorry! I don't understand you


KeyboardInterrupt: ignored

Testing rasa library

In [0]:
import spacy

nlp = spacy.load('en')

doc = nlp("my friend Mary has worked at Google since 2009")

#label_ gives word represantation of label, label gives number of the label

#to get the named entities:
for entity in doc.ents:
  print(entity.text, entity.label_)
  
#to get all the tokens(every word) in the text, as a generator
for item in doc:
  print(item)
  
#to get ancestors of a token - words that are linked semantically:
list(doc[2].ancestors)

Mary PERSON
Google ORG
2009 DATE
my
friend
Mary
has
worked
at
Google
since
2009


[friend, worked]

In [0]:
!pip install rasa_nlu


    100% |████████████████████████████████| 153kB 10.0MB/s 
    100% |████████████████████████████████| 61kB 22.5MB/s 
    100% |████████████████████████████████| 12.6MB 2.7MB/s 
    100% |████████████████████████████████| 665kB 24.1MB/s 
    100% |████████████████████████████████| 51kB 19.9MB/s 
    100% |████████████████████████████████| 81kB 24.4MB/s 
    100% |████████████████████████████████| 3.1MB 9.9MB/s 
    100% |████████████████████████████████| 81kB 24.6MB/s 
    100% |████████████████████████████████| 174kB 30.8MB/s 
    100% |████████████████████████████████| 61kB 23.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/5d/1a/1e/0350bb3df3e74215cd91325344cc86c2c691f5306eb4d22c77
  Stored in directory: /root/.cache/pip/wheels/57/2e/89/11ba83bc08ac30a5e3a6005f0310c78d231b96a270def88ca0
Successfully built simplejson Twisted
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
google-colab 0.0.1a1 has requirement requests~=2.18.0, but

In [0]:
import rasa_nlu

help(rasa_nlu)

Help on package rasa_nlu:

NAME
    rasa_nlu

PACKAGE CONTENTS
    classifiers (package)
    components
    config
    convert
    data_router
    emulators (package)
    evaluate
    extractors (package)
    featurizers (package)
    model
    persistor
    project
    registry
    run
    server
    tokenizers (package)
    train
    training_data (package)
    utils (package)
    version

DATA
    absolute_import = _Feature((2, 5, 0, 'alpha', 1), (3, 0, 0, 'alpha', 0...
    division = _Feature((2, 2, 0, 'alpha', 2), (3, 0, 0, 'alpha', 0), 8192...
    print_function = _Feature((2, 6, 0, 'alpha', 2), (3, 0, 0, 'alpha', 0)...
    unicode_literals = _Feature((2, 6, 0, 'alpha', 2), (3, 0, 0, 'alpha', ...

VERSION
    0.14.2

FILE
    /usr/local/lib/python3.6/dist-packages/rasa_nlu/__init__.py


